In [1]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np

In [15]:
data = pd.read_csv("online_shoppers.csv", encoding = "ISO-8859-1")
data.dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Prepare data

In [16]:
#Make sales column (quantity * price)
data['Sales'] = data['Quantity'] * data['UnitPrice']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [17]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [18]:
date_last_purchase = data['InvoiceDate'].max()
date_last_purchase

Timestamp('2011-12-09 12:50:00')

In [19]:
df = data.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
     'InvoiceNo': 'count',
     'Sales': 'sum'})
df.head()

,InvoiceDate,InvoiceNo,Sales
CustomerID,,,
12346.0,325,2,0.00
12347.0,1,182,4310.00
12348.0,74,31,1797.24
12349.0,18,73,1757.55
12350.0,309,17,334.40


In [21]:
df['SalesPerOrder'] = df.Sales / df.InvoiceNo
df.head()

,InvoiceDate,InvoiceNo,Sales,SalesPerOrder
CustomerID,,,,
12346.0,325,2,0.00,0.000000
12347.0,1,182,4310.00,23.681319
12348.0,74,31,1797.24,57.975484
12349.0,18,73,1757.55,24.076027
12350.0,309,17,334.40,19.670588


In [22]:
df = df.drop(columns = "Sales")
df.head()

,InvoiceDate,InvoiceNo,SalesPerOrder
CustomerID,,,
12346.0,325,2,0.000000
12347.0,1,182,23.681319
12348.0,74,31,57.975484
12349.0,18,73,24.076027
12350.0,309,17,19.670588


In [23]:
df.rename(columns = {'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'SalesPerOrder': 'Monetary'}, inplace = True)
df.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,2,0.000000
12347.0,1,182,23.681319
12348.0,74,31,57.975484
12349.0,18,73,24.076027
12350.0,309,17,19.670588


# Classify Customers Based on Results

Sort data, divide it into quartiles and give customer 1-4 points in each category depending on which quartile their data is in. 



In [26]:
df['F'] = pd.qcut(x =df['Frequency'], q = 4, labels = range (1, 5, 1))
df.head()

,Recency,Frequency,Monetary,F
CustomerID,,,,
12346.0,325,2,0.000000,1
12347.0,1,182,23.681319,4
12348.0,74,31,57.975484,2
12349.0,18,73,24.076027,3
12350.0,309,17,19.670588,1


In [27]:
df['M'] = pd.qcut(x = df['Monetary'], q = 4, labels = range(1, 5, 1))
df.head()

,Recency,Frequency,Monetary,F,M
CustomerID,,,,,
12346.0,325,2,0.000000,1,1
12347.0,1,182,23.681319,4,4
12348.0,74,31,57.975484,2,4
12349.0,18,73,24.076027,3,4
12350.0,309,17,19.670588,1,3


In [28]:
df['R'] = pd.qcut(x = df['Recency'], q = 4, labels = range(4, 0, -1))
df.head()

,Recency,Frequency,Monetary,F,M,R
CustomerID,,,,,,
12346.0,325,2,0.000000,1,1,1
12347.0,1,182,23.681319,4,4,4
12348.0,74,31,57.975484,2,4,2
12349.0,18,73,24.076027,3,4,3
12350.0,309,17,19.670588,1,3,1


In [29]:
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head()

,Recency,Frequency,Monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,325,2,0.000000,1,1,1,3
12347.0,1,182,23.681319,4,4,4,12
12348.0,74,31,57.975484,2,4,2,8
12349.0,18,73,24.076027,3,4,3,10
12350.0,309,17,19.670588,1,3,1,5


# RFM Function

In [30]:
def rfm_score(df):
    if df['RFM'] >= 11:
        return 'High Value Customer'
    elif((df['RFM'] >= 8) and (df['RFM'] < 11)):
        return 'Value Customer'
    elif ((df['RFM'] >= 6) and (df['RFM'] < 8)):
        return 'Potential Value Customer'
    else: 
        return 'Low Value Customer'

In [31]:
df['RFM_level'] = df.apply(rfm_score, axis = 1)
df.head()

,Recency,Frequency,Monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
12346.0,325,2,0.000000,1,1,1,3,Low Value Customer
12347.0,1,182,23.681319,4,4,4,12,High Value Customer
12348.0,74,31,57.975484,2,4,2,8,Value Customer
12349.0,18,73,24.076027,3,4,3,10,Value Customer
12350.0,309,17,19.670588,1,3,1,5,Low Value Customer
